In [1]:
from wmts_extractor.extractor import Extractor
from wmts_extractor.utils import obtain_decision_table

# Arguments

Dictionary with options supported by the `Extractor` module.

In [2]:
args = dict(
    start_datetime=None, # Ignore imagery acquired before start datetime – format DD/MM/YYYY HH:MM:SS
    end_datetime=None,   # Ignore imagery acquired after end datetime – format DD/MM/YYYY HH:MM:SS
    max_cloud=None,      # Max cloud cover in percentage - example: 0.5
    features=None,       # Identify imagery for download by specifying list unique feature identifiers defined in metadata
    overlap=None,        # Minimum percentage overlap - example: 80
    platforms=None,      # List of space-separated platforms - example: ['WorldView-01', 'GeoEye-01']
    max_resolution=None, # Max resolution in meters - example: 0.4
    period_resolution='/cfg/period_resolution_filter.yml', # File location containing weights assigned to a pair time period/resolution
    overwrite=False,     # Overwrite existing output files – otherwise skip download
    max_downloads=None,  # Max compliant downloads for aoi
    dirs=None,           # Path structure
    format='GTIFF',      # Output image format
    options='TILED=YES COMPRESS=LZW', # Output image creation options
    out_path='/images/', # specifies root directory to copy output images
    zoom=18,             # integer value between 1 and maximum zoom level supported by WMTS end point
)

# Configuration

The `config` dictionary specifies identity of Digital Globe WMTS end point along with necessary login 
credentials. It also defines full pathname to OGR-supported file defining one or more area of interest geometries. An example of template configuration dictionary is shown below:

In [3]:
config = dict(
    endpoint=dict(
        name='securewatch',
        credentials=dict(
            username='yolanda.vazquez@sa.catapult.org.uk',
            password=''
        ),
        id='0f96fb54-cbb7-4191-afec-ea6372b311ab', # unique 32-character id assigned by Maxar to access services
        uri='https://securewatch.digitalglobe.com/earthservice/wmtsaccess',
        type='slippy',
        layer='DigitalGlobe:ImageryTileService', 
        format='png', 
        tilematrixset='EPSG:3857', 
        profile='Currency_RGB_Profile'    
    ),
    aoi=dict(
        pathname='/cfg/aoi/aoi_example.shp', # full pathname to OGR supported file encoding one or more Point, Line or Polygon geometries
        name='name'
    )
)

## Period Resolution filter

This filter has been added to fill a recurrent use case where the user needs a filtered list of products under a complex
criteria based on periods of time and resolution.

To do this, we have to design a bi-dimensional table that relates a pair of `time periods` and `resolution` values by 
what we call `weights`. An example of this table can be seen below.

If we pay attention to the first row, we can deduce that for a time period between `2011-01-01` and `2011-01-01` for the
given resolution values (`0.3`, `0.4` and `0.5`) there are some weights assigned with values `11`, `12` and `13`.

This filter contains the maximum number of images we want to obtain as additional condition, given by `number_images`.

**NOTE**: This filter can be used in addition to other filters such as minimum number for overlap (`-o`) and maximum
number of cloud cover (`-c`), but will ignore any other related to time periods (`-s`, `-e`) or resolution (`-r`).

In [4]:
period_resolution = {
    'number_images': 7,
    'resolutions': [0.3, 0.4, 0.5],
    'periods': [
        {
            'name': 'Period 1',
            'date_range': ['1/1/2011', '31/12/2014'],
            'weights': [[11, 12, 12], 
                        [21, 22, 23], 
                        [31, 32, 33]]
        },
        {
            'name': 'Period 2',
            'date_range': ['31/1/2015', '1/12/2018'],
            'weights': [[41, 42, 43],
                        [51, 52, 53],
                        [61, 62, 63],
                        [71, 72, 73]]
        }
    ]
}

obtain_decision_table(period_resolution)

,Period Name,0.3,0.4,0.5
2011-01-01,Period 1,11,12,12
2012-12-31,Period 1,21,22,23
2014-12-31,Period 1,31,32,33
2015-01-31,Period 2,41,42,43
2016-01-25,Period 2,51,52,53
2017-01-18,Period 2,61,62,63
2018-01-12,Period 2,71,72,73


In [5]:
# Replace yml file path with the dictionary itself (both can be used)
args['period_resolution'] = period_resolution

# Create the SecureWatch Extractor object
securewatch = Extractor(config, args)

# Get all tiles under the conditions given above
tiles = securewatch.get_tiles()

# Show the tiles table
tiles

,platform,uid,product,acq_datetime,cloud_cover,resolution,geometry,overlap,Weights,Period Name
0,WorldView-03,708f35be5cdb8810eff1a91f48420951,Pan Sharpened Natural Color,2017-08-28 07:50:19,0.000000,0.3,"POLYGON ((44.92616 12.99229, 44.96102 12.99015...",100.000000,61,Period 2
1,WorldView-02,4e1a9ad33e8e8867f3198b6cbb1906cf,Pan Sharpened Natural Color,2014-03-10 07:41:54,0.038534,0.5,"POLYGON ((44.89797 13.03365, 44.92925 13.02978...",100.000000,23,Period 1
2,WorldView-03,66c9145994bd1dae337d05708f3c2c8c,Pan Sharpened Natural Color,2014-11-04 07:36:45,0.000000,0.3,"POLYGON ((44.95195 12.75269, 44.95204 12.78711...",50.113704,21,Period 1
3,WorldView-03,9a93bbeb4991fa865fb083809cbf0e63,Pan Sharpened Natural Color,2014-10-21 07:16:16,0.011507,0.3,"POLYGON ((44.95209 12.90339, 44.95328 12.90329...",100.000000,21,Period 1
4,GeoEye-01,f0a5b45e09f868a70e004337488674c7,Pan Sharpened Natural Color,2011-10-23 07:22:39,0.007486,0.4,"POLYGON ((44.78526 12.95904, 44.81924 12.95862...",100.000000,12,Period 1
5,WorldView-02,1ffcacf3ebb20241b067a14b4ffd2786,Pan Sharpened Natural Color,2011-03-10 07:58:48,0.029268,0.4,"POLYGON ((44.94652 12.72968, 44.94654 12.75920...",100.000000,12,Period 1
6,GeoEye-01,95603441945ef85d77d7075e6c806eed,Pan Sharpened Natural Color,2011-03-01 07:26:02,0.498521,0.4,"POLYGON ((44.81506 13.01072, 44.85011 13.01000...",100.000000,12,Period 1
